## Example for implementation of Spectrum estimation based on the Order Tracking

My objective is to increase the coverage for machines with unstable operational conditions (load and torque). I have bumped into a crane with an unstable operational speed. For this case, I want to use Order Tracking, allowing further utilization of the already implemented spectrum analysis tools. Later, I think that we can use this technique for other equipment too. 

In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.signal import ellip, firwin, filtfilt, sosfreqz, sosfiltfilt
from scipy.ndimage.filters import uniform_filter1d
from scipy.fftpack import fft, fftfreq, next_fast_len
from scipy.interpolate import interp1d

The magnetic file names

In [ ]:
files = glob('60f5e7b1502edc0001230701/*magnetic*')
files

In [ ]:
def plotFiltResponse(sos, fs):
    w, h = sosfreqz(sos, fs=fs, worN=200000)
    dispCond = w < 20
    plt.figure(figsize=[9.5, 5])
    plt.plot(w[dispCond], abs(h[dispCond]))
    plt.title('Filter frequency response')
    plt.xlabel('Frequency [radians / second]')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.show()

Magnetic file where the engine is on

In [ ]:
filesMagnetic = []
for file in files:
    x = np.load(file)
    if 'sos' not in locals():
        fs = 1 / x[0, 1]
        sosHigh = ellip(8, 1, 40, 2, btype='high', output='sos', fs=fs)
#         plotFiltResponse(sos, fs)
    temp = sosfiltfilt(sosHigh, x[1, :])  
    # looking for a file with activity
    cond1 = (np.percentile(temp, 80) - np.percentile(temp, 20)) > 0.5
    cond2 = x[1, :].mean() < 100
    if (cond1 & cond2):
        plt.figure(figsize=[9.5, 5])
        plt.plot(x[0, :], x[1, :])
        plt.title(file.split('/')[-1])
        plt.show()
        filesMagnetic.append(file)
len(filesMagnetic)

In [ ]:
def getSpeed(xMagnetic, tMagnetic, b=firwin(9, 0.1), isPlot=False):
    # this function implements the speed estimation based on phase zero corssing and 
    # lowpass filtering of the estimated speed
    crossLocs = np.where(np.diff(np.sign(xMagnetic - np.median(xMagnetic))))[0]
    # avoiding fault detection of crossing due to noise
    crossLocs = crossLocs[np.where(np.diff(crossLocs)>30)]
    #avoiding sampling of half-cycle, which leads to leakage
    if crossLocs.shape[0] % 2:
        crossLocs = crossLocs[:-1]
        # locations of the estimated speed
    speedLocs = (crossLocs[:-1] + crossLocs[1:]) / 2
    speedTime = speedLocs / fs
    # could skip estimation of the speed but it is nise to have it
    speed = fs / np.diff(crossLocs) / 2
    # due to the presence of higher harmonics in the signal, we have to filter the estimated speed. 
    # Here, I don't care that the distance between the samples is not constant
    filtSpeed = filtfilt(b, 1, speed, padtype='even')
    
    if isPlot:
        plt.figure(figsize=[9.5, 5])
        plt.plot(speedTime, speed)
        plt.plot(speedTime, filtSpeed)
        plt.legend(['Gross speed estimation', 'Filtered speed estimtaton'])
        plt.xlabel('Time [sec]')
        plt.ylabel('Shaft speed [Cycles / second]')
        plt.plot()
        plt.show()
        
    return filtSpeed, speedTime

In [ ]:
def orderTrack(x, t, speed, speedTime):
    #this function implements the order-tracking
    
    #setting x dimensionality 
    x = np.atleast_2d(x)
    if (x.shape[1] > x.shape[0]):
        x = x.T
    
    # trimming the samples with a non-defined speed 
    cond = (t > speedTime[0]) & (t <speedTime[-1])
    x = x[cond, :]
    t = t[cond]
    
    #interpolation of speed
    interpSpeedFun = interp1d(speedTime, speed, axis=0)
    interpSpeed = interpSpeedFun(t)
     
    #phase in cycle units
    phase = np.cumsum(interpSpeed) * (t[1] - t[0])
    phase -= phase[0]
    # avoiding aliasing by sampling at least as fast as previously
    minDphase = np.min(speed) * (t[1] - t[0])
    # finding the increment for a fast fft calculation - 
    # considers having aggregation points at round shaft speed orders
    dPhase = phase[-1] / next_fast_len(np.ceil(phase[-1] / minDphase).astype(int))
    resampledPhase = np.arange(0, phase[-1], dPhase)
    interpFun = interp1d(phase, x, axis=0)
    resampledX = interpFun(resampledPhase)

    return resampledPhase, resampledX, t, phase, interpSpeed, x

Packing the data of the vibration signals with magnetic signal

In [ ]:
for fileMagnetic in filesMagnetic:
    print(fileMagnetic)
    filesVib = glob(fileMagnetic.split('_plane_')[0] + '*vibration*')
    xVibration = np.concatenate([np.atleast_2d(np.load(file)[1, :]) for file in filesVib]).T
    xVibration -= xVibration.mean(axis=0)
    tVibration = np.load(filesVib[0])[0, :]
    
    x = np.load(fileMagnetic)
    tMagnetic = x[0, :]
    xMagnetic = x[1, :]
    
    speed, speedTime = getSpeed(xMagnetic, tMagnetic, isPlot=True)
    resampledPhase, resampledMagnetic, _, _, _, _ = orderTrack(xMagnetic, tMagnetic, speed, speedTime)
    
    plt.figure(figsize=[9.5, 5])
    
    f = fftfreq(resampledMagnetic.shape[0], d=(resampledPhase[1] - resampledPhase[0]))
    xWind = np.squeeze(resampledMagnetic) * np.hanning(resampledMagnetic.shape[0])
    dft = fft(xWind, n=xWind.shape[0])
    positiveCond = f > 0
    f = f[positiveCond]
    dft = dft[positiveCond]
    dft = np.log(np.abs(dft))
    plt.plot(f, dft)
    
    f = fftfreq(xMagnetic.shape[0], d=(tMagnetic[1] - tMagnetic[0]))
    xWind = np.squeeze(xMagnetic) * np.hanning(xMagnetic.shape[0])
    dft = fft(xWind, n=xWind.shape[0])
    positiveCond = f > 0
    f = f[positiveCond]
    dft = dft[positiveCond]
    dft = np.log(np.abs(dft))
    plt.plot(f / speed.mean(), dft)
    
    plt.xlabel('Shaft Speed Order [-] / Frequency normalized by mean speed')
    plt.ylabel('DFT-Absolute Value - log scale')
    plt.xlim([0, 10])
    plt.xticks(np.arange(11))
    plt.grid()
    plt.show()
    
    resampledPhase, resampledVibration, t, _, _, _ = orderTrack(xVibration, tVibration, speed, speedTime)
    
    for iSig in np.arange(resampledVibration.shape[1]):
        plt.figure(figsize=[9.5, 5])

        f = fftfreq(resampledVibration.shape[0], d=(resampledPhase[1] - resampledPhase[0]))
        xWind = np.squeeze(resampledVibration[:, iSig]) * np.hanning(resampledVibration.shape[0])
        dft = fft(xWind, n=xWind.shape[0])
        positiveCond = f > 0
        f = f[positiveCond]
        dft = dft[positiveCond]
        dft = np.log(np.abs(dft))
        plt.plot(f, dft)

        f = fftfreq(xVibration.shape[0], d=(tVibration[1] - tVibration[0]))
        xWind = np.squeeze(xVibration[:, iSig]) * np.hanning(tVibration.shape[0])
        dft = fft(xWind, n=xWind.shape[0])
        positiveCond = f > 0
        f = f[positiveCond]
        dft = dft[positiveCond]
        dft = np.log(np.abs(dft))
        plt.plot(f / speed.mean(), dft)

        plt.xlabel('Shaft Speed Order [-] / Frequency normalized by mean speed')
        plt.ylabel('DFT-Absolute Value - log scale')
        plt.xlim([0, 30])
        plt.xticks(np.arange(31))
        plt.grid()
        plt.show()

For non-stationary cases, we can see an improvement in the spectrum elements' decomposition. Besides that, we can see the following improvements:
1. Reduction in the dc noise (probably from the hardware). Why?
2. Reduction in the leakage (this one is yet pending improvement). Have been seen in multiple locations (see the last magnetic signal spectrum). This probably happens due to the trimming of the signal at the precise multiplication of the cycle.
3. Concentration of the energy in the integration points, located at the harmonics of the shaft speed order.

Points to improve:
1. Trimming of the signal where the machine is off.
2. figure out why we have DC.